# Introduction to Natrual Language Processing: Sentiment Analysis

### Import Libaries

In [ ]:
import nltk
import sklearn
import wordcloud

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Download Data

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [ ]:
!tar -xzf aclImdb_v1.tar.gz

### Load Data

In [ ]:
# Load Training Data

import pathlib

train_pos = pathlib.Path.cwd() / "aclImdb" / "train" / "pos"
train_neg = pathlib.Path.cwd() / "aclImdb" / "train" / "neg"

negative_texts = []
i = 0
for file_path in train_neg.glob('*.txt'):
    if i < 5000:
        with open(file_path, 'r') as f:
            negative_texts.append(f.read())
        i += 1
negative_labels = [0] * len(negative_texts)

positive_texts = []
i = 0
for file_path in train_pos.glob('*.txt'):
    if i < 5000:
        with open(file_path, 'r') as f:
            positive_texts.append(f.read())
        i += 1
positive_labels = [1] * len(positive_texts)

texts = positive_texts + negative_texts
labels = positive_labels + negative_labels

### Tokenization

In [ ]:
from nltk.tokenize import word_tokenize

#### Split Words

In [ ]:
tokenized_texts = [word_tokenize(text) for text in texts]

In [ ]:
tokenized_texts[0]

#### Create Vocabulary

In [ ]:
vocab = set([token for text in tokenized_texts for token in text])

In [ ]:
len(vocab)

#### Lowercase and Remove Punctuation

In [ ]:
vocab = set([token.lower() for token in vocab if token.isalnum()])

In [ ]:
len(vocab)

In [ ]:
tokenized_texts = [[token.lower() for token in tokens  if token.isalnum()] for tokens in tokenized_texts]

In [ ]:
len(tokenized_texts)

In [ ]:
tokenized_texts[0]

### Stemming

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
stemmer.stem('movies')

### Lemmatization

In [ ]:
from nltk.corpus import wordnet

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [ ]:
wnl.lemmatize('movies')

In [ ]:
vocab = set([wnl.lemmatize(token) for token in vocab if wordnet.synsets(token)])

In [ ]:
len(vocab)

In [ ]:
lemmatized_texts = [[wnl.lemmatize(token) for token in tokens  if wordnet.synsets(token)] for tokens in tokenized_texts]

In [ ]:
lemmatized_texts[0]

### Stop Word Removal

#### Find most frequent tokens

In [ ]:
all_tokens = [token for tokens in lemmatized_texts for token in tokens]

In [ ]:
freq = nltk.FreqDist(all_tokens)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 5))  
freq.plot(50, cumulative=False)

In [ ]:
stop_words = freq.most_common()[:50]
stop_words = list(zip(*stop_words))[0]
print(stop_words)

#### Get stopwords from NLTK list

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
stop_words = set(stop_words).union(set(stopwords.words('english')))

#### Remove stopwords from vocab

In [ ]:
vocab = set([token for token in vocab if token not in stop_words])

In [ ]:
len(vocab)

### Count Vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(vocabulary=list(vocab))

In [ ]:
count_vectors = vectorizer.fit_transform(texts)

In [ ]:
count_vectors.shape

### Train Model

In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(count_vectors, labels, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0).fit(X, y)

### Load Test Data

In [ ]:
test_pos = pathlib.Path.cwd() / "aclImdb" / "test" / "pos"
test_neg = pathlib.Path.cwd() / "aclImdb" / "test" / "neg"

test_negative_texts = []
i = 0
for file_path in test_neg.glob('*.txt'):
    if i < 1000:
        with open(file_path, 'r') as f:
            test_negative_texts.append(f.read())
        i += 1
test_negative_labels = [0] * len(test_negative_texts)

test_positive_texts = []
i = 0
for file_path in test_pos.glob('*.txt'):
    if i < 1000:
        with open(file_path, 'r') as f:
            test_positive_texts.append(f.read())
        i += 1
test_positive_labels = [1] * len(test_positive_texts)

test_texts = test_positive_texts + test_negative_texts
test_labels = test_positive_labels + test_negative_labels

### Evaluate Accuracy

In [ ]:
X_test = vectorizer.transform(test_texts)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Test Accuracy: %.2f%%" % sklearn.metrics.accuracy_score(test_labels, y_pred))

### Feature Importance

In [ ]:
importance = model.coef_[0]

In [ ]:
positive_words = dict()
for i, word in enumerate(vocab):
    positive_words[word] = importance[i]

In [ ]:
cloud = wordcloud.WordCloud().generate_from_frequencies(positive_words)
plt.figure(figsize=(16,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
negative_words = dict()
for i, word in enumerate(vocab):
    negative_words[word] = -importance[i]

In [ ]:
cloud = wordcloud.WordCloud().generate_from_frequencies(negative_words)
plt.figure(figsize=(16,12))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()